<a href="https://www.kaggle.com/code/riyakeshwani/generate-leak-data?scriptVersionId=112778778" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on https://www.kaggle.com/wuliaokaola/ashrae-maybe-this-can-make-public-lb-some-useful public kernel:

the idea is to reconstitute from the public kernels a submission file with the data that were scraped and believed to be the true values on the test set. this method was validated by Kaggle / sponsors.

this notebook requires to download several files from the public kernels.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import pickle
import numpy as np
import pandas as pd
import random as rn
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime

In [2]:
test = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/test.csv', index_col=0, parse_dates = ['timestamp'])
building = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/building_metadata.csv', usecols=['site_id', 'building_id'])

In [3]:
test = test.merge(building, left_on = "building_id", right_on = "building_id", how = "left")

In [4]:
submission = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/sample_submission.csv')

In [5]:
# give a numeric value to all submissions that is really unlikely:
submission['meter_reading'] = -9999.99

* Site 0

Thanks [@yamsam](https://www.kaggle.com/yamsam/) and his great kernel.

https://www.kaggle.com/yamsam/new-ucf-starter-kernel

Download submission_ucf_replaced.csv from https://www.kaggle.com/yamsam/new-ucf-starter-kernel

In [6]:
site_0 = pd.read_csv('/kaggle/input/new-ucf-starter-kernel/submission_ucf_replaced.csv', index_col=0)
submission.loc[test[test['site_id']==0].index, 'meter_reading'] = site_0['meter_reading']
del site_0
gc.collect()

49

* Site 1

Thanks [@mpware](https://www.kaggle.com/mpware) and his great kernel.

https://www.kaggle.com/mpware/ucl-data-leakage-episode-2


Download site1.pkl from https://www.kaggle.com/mpware/ucl-data-leakage-episode-2

In [7]:
with open('/kaggle/usr/lib/ucl_data_leakage_episode_2/site1.pkl', 'rb') as f:
    site_1 = pickle.load(f)
site_1 = site_1[site_1['timestamp'].dt.year > 2016]

In [8]:
t = test[['building_id', 'meter', 'timestamp']]
t['row_id'] = t.index
site_1 = site_1.merge(t, left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = "left")
site_1 = site_1[['meter_reading_scraped', 'row_id']].set_index('row_id').dropna()
submission.loc[site_1.index, 'meter_reading'] = site_1['meter_reading_scraped']
del site_1
gc.collect()

0

* Site 2

Thanks [@pdnartreb](https://www.kaggle.com/pdnartreb) and his great dataset.

https://www.kaggle.com/pdnartreb/asu-buildings-energy-consumption

In [9]:
site_2 = pd.read_csv('/kaggle/input/asu-buildings-energy-consumption/asu_2016-2018.csv', parse_dates = ['timestamp'])
site_2 = site_2[site_2['timestamp'].dt.year > 2016]

In [10]:
site_2

,timestamp,building_id,meter,meter_reading
2530312,2017-01-01 00:00:00,156,0,104.96
2530313,2017-01-01 00:00:00,157,0,32.92
2530314,2017-01-01 00:00:00,158,0,10.97
2530315,2017-01-01 00:00:00,159,0,8.70
2530316,2017-01-01 00:00:00,160,0,10.08
...,...,...,...,...
7593587,2018-12-31 23:00:00,287,3,0.00
7593588,2018-12-31 23:00:00,288,0,26.53
7593589,2018-12-31 23:00:00,289,0,22.94
7593590,2018-12-31 23:00:00,290,0,102.04


In [11]:
t = test[['building_id', 'meter', 'timestamp']]
t['row_id'] = t.index
site_2 = site_2.merge(t, left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = "left")
site_2 = site_2[['meter_reading', 'row_id']].set_index('row_id').dropna()
submission.loc[site_2.index, 'meter_reading'] = site_2['meter_reading']
del site_2
gc.collect()

21

* Site 4

Download file from https://www.kaggle.com/serengil/ucb-data-leakage-site-4-81-buildings/output. Please make sure that you are downloading from one of the last commit since the first commits seems to be wrong.

In [12]:
site_4 = pd.read_csv('/kaggle/input/ucb-data-leakage-site-4-81-buildings/site4.csv', parse_dates = ['timestamp'])
site_4 = site_4[site_4['timestamp'].dt.year > 2016]
site_4['meter']=0

site_4.columns = ['building_id', 'timestamp', 'meter_reading', 'meter']

In [13]:
site_4.head()

,building_id,timestamp,meter_reading,meter
8784,641,2017-01-01 00:00:00,298.120,0
8785,641,2017-01-01 01:00:00,305.534,0
8786,641,2017-01-01 02:00:00,280.643,0
8787,641,2017-01-01 03:00:00,277.854,0
8788,641,2017-01-01 04:00:00,280.613,0


In [14]:
site_4 = site_4.merge(t, left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = "left")

In [15]:
site_4 = site_4[['meter_reading', 'row_id']].dropna().set_index('row_id')

In [16]:
site_4.shape

(1415273, 1)

In [17]:
# site_4 = site_4[['meter_reading', 'row_id']].set_index('row_id').dropna()
submission.loc[site_4.index, 'meter_reading'] = site_4['meter_reading']

* Site 15

Download file site15_leakage from https://www.kaggle.com/pp2file/ashrae-site15-cornell/output.

In [18]:
site_15 = pd.read_csv('/kaggle/input/ashrae-site15-cornell/site15_leakage.csv', parse_dates = ['timestamp'])
site_15 = site_15[site_15['timestamp'].dt.year > 2016]
site_15

,timestamp,meter,meter_reading,building_id,score
0,2017-01-01 00:00:00,0.0,27.650000,1429,1.0
1,2017-01-01 00:00:00,2.0,278.528900,1426,1.0
2,2017-01-01 01:00:00,0.0,28.674999,1429,1.0
3,2017-01-01 01:00:00,2.0,373.365875,1426,1.0
4,2017-01-01 02:00:00,0.0,27.200001,1429,1.0
...,...,...,...,...,...
2476696,2018-12-31 19:00:00,0.0,184.524994,1363,1.0
2476697,2018-12-31 20:00:00,0.0,183.600006,1363,1.0
2476698,2018-12-31 21:00:00,0.0,178.475006,1363,1.0
2476699,2018-12-31 22:00:00,0.0,179.725006,1363,1.0


In [19]:
submission.reset_index().to_feather('leak012345_001.feather')

In [20]:
site_15 = site_15[['building_id', 'timestamp', 'meter_reading', 'meter']]
site_15 = site_15.merge(t, left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = "left")
site_15 = site_15[['meter_reading', 'row_id']].dropna().set_index('row_id')
site_15.shape

(2476701, 1)

In [21]:
submission.loc[site_15.index, 'meter_reading'] = site_15['meter_reading']